### Gradio UI from Hugging Face ###

In [1]:
#imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

c:\Python\llm_engineering\llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Loads environment variables in hidden .env file
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY') #OPENAI_API_KEY is in Path variable on local machine
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

#Connect
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [3]:
system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [
        {"role":"system","content":system_message},
        {"role":"user","content":prompt}
    ]
    completion = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )
    return completion.choices[0].message.content

In [4]:
message_gpt("Tell me today's date")

"Today's date is October 9, 2023."

## Simple UI ##

In [5]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [6]:
shout("hi there")

Shout has been called with input hi there


'HI THERE'

In [7]:
# Gradio 
view = gr.Interface(fn=shout, inputs = "textbox", outputs = "textbox").launch()
#users can flag results 

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [8]:
gr.Interface(fn=shout, inputs = "textbox", outputs = "textbox", allow_flagging="never").launch(share=True)
#share = True will actually give public URL to use for folks to test

c:\Python\llm_engineering\llms\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://10ce41e6f727bb01e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
view = gr.Interface(
    fn = message_gpt,
    inputs = [gr.Textbox(label="Your message:", lines = 6)],
    outputs = [gr.Textbox(label="Response:", lines=8)],
    allow_flagging = "never"
)

view.launch()

c:\Python\llm_engineering\llms\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [10]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs = [gr.Textbox(label="Your message:")],
    outputs = [gr.Markdown(label="Response")],
    allow_flagging = "never"
)
view.launch()

c:\Python\llm_engineering\llms\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [11]:
#Streaming
def stream_gpt(prompt):
    messages = [
        {"role":"system","content":system_message},
        {"role":"user","content":prompt}
    ]
    stream = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    #add in chunk to result and then yield total result - else chunk will appear then disappear
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [12]:
view = gr.Interface(
    fn = stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response")],
    allow_flagging="never"
)
view.launch()

c:\Python\llm_engineering\llms\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [13]:
#stream claude
def stream_claude(prompt):
    result = claude.messages.stream(
        model = "claude-3-haiku-20240307",
        max_tokens = 1000,
        temperature = 0.7,
        system=system_message,
        messages=[
            {"role":"user","content":prompt}
        ]
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [14]:
view = gr.Interface(
    fn = stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response")],
    allow_flagging="never"
)
view.launch()

c:\Python\llm_engineering\llms\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


## Create a dropdown to switch between models in gradio ##

In [20]:
def stream_model(prompt,model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Uknown Model")
    for chunk in result:
        yield chunk

In [21]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT","Claude"],label="Select model")],
    outputs=[gr.Markdown(label="Response")],
    allow_flagging="never"
)
view.launch()

c:\Python\llm_engineering\llms\Lib\site-packages\gradio\interface.py:403: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


##  Build company brochure generator ##

In [22]:
class Website:
    url: str
    title: str 
    text: str

    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body,'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n",strip=True) 
    
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n {self.text}\n\n"

In [23]:
system_prompt = "You are an assistant that analyzes the contents of a company website landing page \
    and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [24]:
def stream_brochure(company_name, url, model):
    prompt=f"Please generate a company brochure for {company_name}. Here is their landing page: \n"
    prompt += Website(url).get_contents()
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Uknown Model")
    for chunk in result:
        yield chunk

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[gr.Textbox(label="Company Name:"), 
            gr.Textbox(label="Landing Page Url:"),
            gr.Dropdown(["GPT","Claude"],label="Select model")],
    outputs=[gr.Markdown(label="Response")],
    allow_flagging="never"
)
view.launch()